In [18]:
!pip install evaluate
import pandas as pd
import numpy as np
import torch
from datasets import Dataset, DatasetDict
import evaluate
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification
from tqdm.auto import tqdm
import os
import glob
import ast

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Используется устройство: {device}")

Используется устройство: cuda


In [19]:
DATA_DIR = "/kaggle/input/avito-task-segment/"
TOTAL_TRAIN_SAMPLES = 3_000_000 
MODEL_CHECKPOINT = "Geotrend/distilbert-base-ru-cased"
MAX_LEN = 128
RANDOM_SEED = 42

print("Этап 0: Загрузка всех данных в Pandas DataFrame...")
train_files = glob.glob(f"{DATA_DIR}train_*.parquet")
df_list = []
samples_per_file = TOTAL_TRAIN_SAMPLES // len(train_files) if train_files else 0
for file in tqdm(train_files, desc="Чтение Parquet файлов"):
    df = pd.read_parquet(file)
    sample_df = df.sample(n=min(samples_per_file, len(df)), random_state=RANDOM_SEED)
    df_list.append(sample_df)
full_df = pd.concat(df_list, ignore_index=True)
print(f"Загружено {len(full_df)} строк.")

print("\nЭтап 0.5: Очистка и фильтрация данных...")
initial_size = len(full_df)
full_df.drop_duplicates(subset=['sentence_without_spaces'], inplace=True, keep='first')
print(f"Удалено {initial_size - len(full_df)} дубликатов.")

initial_size = len(full_df)
full_df = full_df[full_df['true_positions'].str.len() > 0]
print(f"Отфильтровано {initial_size - len(full_df)} строк без пробелов. Осталось {len(full_df)} ПОЛЕЗНЫХ строк.")

full_train_dataset_raw = Dataset.from_pandas(full_df)
del full_df, df_list
def create_char_level_labels(example):
    text_no_spaces = example["sentence_without_spaces"]
    true_positions = example["true_positions"] 
    
    if not isinstance(text_no_spaces, str) or not text_no_spaces:
        return {'tokens': [], 'labels': []}
    
    tokens = list(text_no_spaces)
    labels = [0] * len(tokens)
    
    for pos in true_positions:
        if 0 < pos <= len(labels):
            labels[pos - 1] = 1
    return {'tokens': tokens, 'labels': labels}

split_dataset = full_train_dataset_raw.train_test_split(test_size=0.1, seed=RANDOM_SEED)
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True, max_length=MAX_LEN)
    labels = []
    for i, label in enumerate(examples["labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None: label_ids.append(-100)
            elif word_idx != previous_word_idx: label_ids.append(label[word_idx])
            else: label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

print("\nЭтап 1: Создание СИМВОЛЬНЫХ токенов и меток...")
processed_split = split_dataset.map(create_char_level_labels, remove_columns=split_dataset["train"].column_names)
print("\nЭтап 2: Токенизация и выравнивание меток...")
final_dataset = processed_split.map(tokenize_and_align_labels, batched=True)
final_dataset['validation'] = final_dataset.pop('test')

print("\nДанные готовы.")
print(f"Размер обучающей выборки: {len(final_dataset['train'])}")
print(f"Размер валидационной выборки: {len(final_dataset['validation'])}")

Этап 0: Загрузка всех данных в Pandas DataFrame...


Чтение Parquet файлов:   0%|          | 0/28 [00:00<?, ?it/s]

Загружено 2901223 строк.

Этап 0.5: Очистка и фильтрация данных...
Удалено 733777 дубликатов.
Отфильтровано 59583 строк без пробелов. Осталось 2107863 ПОЛЕЗНЫХ строк.

Этап 1: Создание СИМВОЛЬНЫХ токенов и меток...


Map:   0%|          | 0/1897076 [00:00<?, ? examples/s]

Map:   0%|          | 0/210787 [00:00<?, ? examples/s]


Этап 2: Токенизация и выравнивание меток...


Map:   0%|          | 0/1897076 [00:00<?, ? examples/s]

Map:   0%|          | 0/210787 [00:00<?, ? examples/s]


Данные готовы.
Размер обучающей выборки: 1897076
Размер валидационной выборки: 210787


In [20]:
print("\nПримеры из обучающей выборки (первые 5 строк):")
for i in range(5):
    example = final_dataset["train"][i]
    print(f"Пример {i+1}: Токены = {example['tokens']}, Метки = {example['labels']}")



Примеры из обучающей выборки (первые 5 строк):
Пример 1: Токены = ['С', 'е', 'н', 'с', 'о', 'р', 'M', 'u', 'l', 't', 'i', 't', 'o', 'u', 'c', 'h', '.'], Метки = [-100, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]
Пример 2: Токены = ['Т', 'а', 'к', 'ж', 'е', 'о', 'т', 'д', 'а', 'м', 'п', 'о', 'л', 'н', 'ы', 'й', 'к', 'о', 'м', 'п', 'л', 'е', 'к', 'т', 'и', 'ч', 'е', 'х', 'о', 'л', 'в', 'п', 'о', 'д', 'а', 'р', 'о', 'к', '.'], Метки = [-100, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, -100]
Пример 3: Токены = ['К', 'в', 'а', 'р', 'т', 'и', 'р', 'а', 'в', 'п', 'р', 'е', 'д', 'ч', 'и', 'с', 'т', 'о', 'в', 'о', 'й', 'о', 'т', 'д', 'е', 'л', 'к', 'е', '.'], Метки = [-100, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, -100]
Пример 4: Токены = ['в', 'с', 'ё', 'э', 'т', 'о', 'п', 'р', 'и', 'х', 'о', 'р', 'а', 'ш', 'и', 'в', 'а', 'н', 'и', 'е', 'и', 'т', 'щ', 'е', 'с', '

In [21]:
from collections import Counter
import numpy as np

print("--- Вычисляем веса классов для борьбы с дисбалансом ---")

labels_counter = Counter()
for item in tqdm(final_dataset["train"], desc="Подсчет меток"):
    labels_counter.update(item['labels'])

del labels_counter[-100]

count_no_space = labels_counter[0]
count_space = labels_counter[1]
total = count_no_space + count_space

print(f"Всего меток 'NO_SPACE' (0): {count_no_space}")
print(f"Всего меток 'SPACE' (1):    {count_space}")

weight_no_space = total / (2 * count_no_space)
weight_space = total / (2 * count_space)

class_weights = torch.tensor([weight_no_space, weight_space], dtype=torch.float).to(device)

print(f"\nВычисленные веса: [NO_SPACE: {weight_no_space:.2f}, SPACE: {weight_space:.2f}]")
print("Это означает, что ошибка на метке 'SPACE' будет 'штрафоваться' сильнее.")

--- Вычисляем веса классов для борьбы с дисбалансом ---


Подсчет меток:   0%|          | 0/1897076 [00:00<?, ?it/s]

Всего меток 'NO_SPACE' (0): 73872622
Всего меток 'SPACE' (1):    12503960

Вычисленные веса: [NO_SPACE: 0.58, SPACE: 3.45]
Это означает, что ошибка на метке 'SPACE' будет 'штрафоваться' сильнее.


In [22]:
!pip install seqeval

In [ ]:
from transformers import EarlyStoppingCallback
import torch.nn as nn

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fct = nn.CrossEntropyLoss(weight=class_weights)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

model = AutoModelForTokenClassification.from_pretrained(MODEL_CHECKPOINT, num_labels=2)
model.to(device)

metric = evaluate.load("seqeval")
label_names = ["NO_SPACE", "SPACE"]
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    true_predictions = [[label_names[p] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    true_labels = [[label_names[l] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    results = metric.compute(predictions=true_predictions, references=true_labels, zero_division=0)
    return {"precision": results["overall_precision"], "recall": results["overall_recall"], "f1": results["overall_f1"], "accuracy": results["overall_accuracy"]}

training_args = TrainingArguments(
    output_dir=f"./results-{MODEL_CHECKPOINT.replace('/', '_')}",
    logging_strategy="epoch",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    num_train_epochs=4,
    weight_decay=0.01,
    fp16=True,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    load_best_model_at_end=True,
    metric_for_best_model="eval_f1",
    report_to="none",
    save_total_limit=1
)

data_collator = DataCollatorForTokenClassification(tokenizer)

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=final_dataset["train"],
    eval_dataset=final_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

print(f"\nНачинаем fine-tuning модели '{MODEL_CHECKPOINT}' с использованием весов классов...")
trainer.train()
print("\nОбучение завершено.")

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at Geotrend/distilbert-base-ru-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_36/1726884055.py:45: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead.
  trainer = CustomTrainer(



Начинаем fine-tuning модели 'Geotrend/distilbert-base-ru-cased' с использованием весов классов...


Epoch,Training Loss,Validation Loss


In [ ]:
import torch.nn.functional as F

def debug_restore_positions(text, model, tokenizer, device, threshold=0.5):
    print(f"--- Отладка для строки: '{text}' ---")
    if not isinstance(text, str) or not text.strip():
        return []

    tokens = list(text)
    inputs = tokenizer(
        tokens,
        is_split_into_words=True,
        return_tensors="pt",
        max_length=MAX_LEN,
        padding="max_length",
        truncation=True
    )
    word_ids_map = inputs.word_ids()
    inputs = {k: v.to(device) for k, v in inputs.items()}

    model.eval()
    with torch.no_grad():
        logits = model(**inputs).logits[0]
    probabilities = F.softmax(logits, dim=1).cpu().numpy()
    predictions_argmax = torch.argmax(logits, dim=1).cpu().numpy()

    print("Индекс | Символ | Логиты (NO_SPACE, SPACE) | Вероятности (NO, YES) | Argmax | Решение (>= порога)")
    print("-" * 80)
    
    positions = []
    
    for i, word_idx in enumerate(word_ids_map):
        if word_idx is not None:
            char = tokens[word_idx]
            logit_vals = logits[i].cpu().numpy()
            prob_vals = probabilities[i]
            argmax_val = predictions_argmax[i]
            decision = "ДА" if prob_vals[1] >= threshold else "НЕТ"
            
            print(f"{i:<6} | {char:<6} | [{logit_vals[0]:>6.2f}, {logit_vals[1]:>6.2f}]      | [{prob_vals[0]:.2f}, {prob_vals[1]:.2f}]            | {argmax_val:<6} | {decision}")

            if decision == "ДА":
                positions.append(word_idx + 1)
                
    final_positions = sorted(list(set(positions)))
    print(f"\nИтоговый результат для порога {threshold}: {final_positions}")
    print("-" * 80 + "\n")
    return final_positions
    
some_test_sentences = [
    "приветмир",
    "продамгараж",
    "книгавхорошемсостоянии",
    "надоподумать"
]

for sentence in some_test_sentences:
    _ = debug_restore_positions(sentence, trainer.model, tokenizer, device, threshold=0.5)

In [ ]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import ast
import glob
import torch
import os

def restore_positions_char_level(text, model, tokenizer, device, max_len=128):
    """Предсказывает позиции пробелов для одной строки текста НА УРОВНЕ СИМВОЛОВ."""
    if not isinstance(text, str) or not text.strip():
        return []

    tokens = list(text)
    inputs = tokenizer(
        tokens,
        is_split_into_words=True,
        return_tensors="pt",
        max_length=max_len,
        padding="max_length",
        truncation=True
    )

    word_ids_map = inputs.word_ids()
    inputs = {k: v.to(device) for k, v in inputs.items()}

    model.eval()
    with torch.no_grad():
        logits = model(**inputs).logits

    predictions = torch.argmax(logits, dim=2)[0].cpu().numpy()
    
    positions = []
    for i, word_idx in enumerate(word_ids_map):
        if word_idx is not None and predictions[i] >= 0.6:
            positions.append(word_idx + 1)
            
    return sorted(list(set(positions)))


def calculate_f1(true_pos, pred_pos):
    if isinstance(true_pos, str):
        try: true_pos = ast.literal_eval(true_pos)
        except: true_pos = []

    true_set = set(true_pos)
    pred_set = set(pred_pos)

    if not pred_set and not true_set: return 1.0

    tp = len(true_set.intersection(pred_set))
    precision = tp / len(pred_set) if pred_set else 0.0
    recall = tp / len(true_set) if true_set else 0.0

    if precision + recall == 0: return 0.0
    return 2 * (precision * recall) / (precision + recall)

print("--- ЭТАП 1: Оценка F1-меры на локальном тестовом наборе ---")
LOCAL_TEST_DIR = "/kaggle/input/avito-task-segment/"
EVAL_SAMPLE_SIZE = 20000 
test_files = glob.glob(f"{LOCAL_TEST_DIR}test_*.parquet")
if not test_files:
    print(f"Предупреждение: В папке '{LOCAL_TEST_DIR}' не найдено файлов test_*.parquet. Пропускаем оценку.")
else:
    try:
        first_test_file = test_files[0]
        print(f"Для быстрой оценки будет использован один файл: {os.path.basename(first_test_file)}")
        local_test_df = pd.read_parquet(first_test_file)

        sample_size = min(EVAL_SAMPLE_SIZE, len(local_test_df))
        eval_sample_df = local_test_df.sample(n=sample_size, random_state=42)
        print(f"Используем сэмпл из {len(eval_sample_df)} строк для оценки F1.")

        texts_to_eval = eval_sample_df['sentence_without_spaces'].fillna('').tolist()

        local_predicted_positions = [
            restore_positions_char_level(text, trainer.model, tokenizer, device, max_len=MAX_LEN)
            for text in tqdm(texts_to_eval, desc="Оценка F1 (инференс)")
        ]

        f1_scores = [
            calculate_f1(true_pos, pred_pos)
            for true_pos, pred_pos in zip(eval_sample_df['true_positions'], local_predicted_positions)
        ]

        average_f1 = np.mean(f1_scores)
        print(f"\n✅ Средняя F1-мера на СЭМПЛЕ локального теста: {average_f1:.4f}")

    except Exception as e:
        print(f"❌ ОШИБКА на ЭТАПЕ 1: Не удалось выполнить оценку. Детали: {e}")


print("\n--- ЭТАП 2: Генерация файла submission.csv для отправки ---")
SUBMISSION_SOURCE_PATH = "/kaggle/input/avito-task-segment/submission.parquet"
try:
    submission_source_df = pd.read_parquet(SUBMISSION_SOURCE_PATH)
    print(f"Загружено {len(submission_source_df)} записей для генерации сабмита.")

    texts_to_submit = submission_source_df['text_no_spaces'].fillna('').tolist()

    final_predicted_positions = [
        restore_positions_char_level(text, trainer.model, tokenizer, device, max_len=MAX_LEN)
        for text in tqdm(texts_to_submit, desc="Генерация сабмита")
    ]

    final_submission_df = pd.DataFrame({
        'id': submission_source_df['id'],
        'predicted_positions': [str(p) for p in final_predicted_positions]
    })
    final_submission_df.to_csv('submission.csv', index=False)

    print("\n✅ Файл submission.csv успешно создан!")
    print("Пример содержимого:")
    print(final_submission_df.head())

except FileNotFoundError:
    print(f"❌ ОШИБКА на ЭТАПЕ 2: Файл для сабмита не найден по пути '{SUBMISSION_SOURCE_PATH}'")
except Exception as e:
    print(f"❌ ОШИБКА на ЭТАПЕ 2: Не удалось обработать файл. Детали: {e}")

In [ ]:
print(final_submission_df.head(40))
